In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
#edu_tp1 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
#edad_tp1 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
#vistas_tp1 = pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv')
#avisos_tp1 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
#post_tp1 = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv')

In [3]:
#post_hasta_abril = pd.read_csv('fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')
#edu_hasta_abril = pd.read_csv('fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
#edad_hasta_abril = pd.read_csv('fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv')
#avisos_hasta_abril = pd.read_csv('fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
#vistas_hasta_abril = pd.read_csv('fiuba_hasta_15_abril/fiuba_3_vistas.csv')

In [4]:
#edu_desp_abril = pd.read_csv('Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')
#edad_desp_abril = pd.read_csv('Fiuba desde 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')
#avisos_desp_abril = pd.read_csv('Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv')
#vistas_desp_abril = pd.read_csv('Fiuba desde 15 Abril/fiuba_3_vistas.csv')

In [5]:
#avisos_faltantes = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

- Concateno todos los set de vistas

In [6]:
vistas_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_3_vistas.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_3_vistas.csv')])
vistas_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18368813 entries, 0 to 11648229
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 560.6+ MB


In [7]:
def normalize_str(date_str):
    return (date_str[:10]+' '+date_str[11:19])
    
vistas_totales['timestamp'] = vistas_totales['timestamp'].apply(normalize_str)
vistas_totales['timestamp'] = pd.to_datetime(vistas_totales['timestamp'])

vistas_totales.drop_duplicates(inplace=True)

vistas_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16323504 entries, 0 to 11648229
Data columns (total 3 columns):
idAviso         int64
timestamp       datetime64[ns]
idpostulante    object
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 498.2+ MB


In [8]:
vistas_totales.rename(columns={'idAviso':'idaviso'}, inplace=True)

In [9]:
vistas_totales.head()

,idaviso,timestamp,idpostulante
0,1111780242,2018-02-23 13:38:13,YjVJQ6Z
1,1112263876,2018-02-23 13:38:14,BmVpYoR
2,1112327963,2018-02-23 13:38:14,wVkBzZd
3,1112318643,2018-02-23 13:38:17,OqmP9pv
4,1111903673,2018-02-23 13:38:18,DrpbXDP


- Calculo la fecha de la ultima visita del usuario a cada anuncio

In [10]:
ultima_vista = vistas_totales.sort_values(by='timestamp', ascending=True).drop_duplicates(subset=['idaviso', 'idpostulante'], keep='last') #me quedo con la ultima vista
ultima_vista.sort_values(by='idaviso', ascending=True, inplace=True)
ultima_vista.reset_index(drop=True, inplace=True)
ultima_vista.rename(columns={'timestamp':'ultima_vista'}, inplace=True)
ultima_vista.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         int64
ultima_vista    datetime64[ns]
idpostulante    object
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 196.8+ MB


In [11]:
ultima_vista.head()

,idaviso,ultima_vista,idpostulante
0,18,2018-04-27 16:06:28,BolNL
1,48375,2018-04-19 10:53:17,RwVdKR
2,169730,2018-04-16 19:55:45,Y0qD8J
3,169730,2018-04-29 14:36:03,dVQBxY
4,169730,2018-04-30 07:28:13,rzRElb


- Calculo cantidad de vistas de cada usuario a cada anuncio

In [12]:
vistas_totales_count = vistas_totales.groupby(by=['idaviso', 'idpostulante']).count()
vistas_totales_count.reset_index(drop=False, inplace=True)
vistas_totales_count.rename(columns={'timestamp':'veces_visto'}, inplace=True)
vistas_totales_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
veces_visto     int64
dtypes: int64(2), object(1)
memory usage: 196.8+ MB


In [13]:
vistas_totales_count.head(5)

,idaviso,idpostulante,veces_visto
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


- Concateno toda la info de educacion

In [14]:
edu_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')])
edu_totales.drop_duplicates(inplace=True)

edu_totales = edu_totales.rename(columns={'nombre':'estudios'})
edu_totales['estudios'] = edu_totales['estudios'].astype('category')
edu_totales['estado'] = edu_totales['estado'].astype('category')

edu_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693641 entries, 0 to 397508
Data columns (total 3 columns):
idpostulante    693641 non-null object
estudios        693641 non-null category
estado          693641 non-null category
dtypes: category(2), object(1)
memory usage: 11.9+ MB


In [15]:
# Creo nuevas columnas indicando el nivel del estudio y del estado, para filtrar mas facilmente luego

edu_estado_list = ['Abandonado','En Curso', 'Graduado']
edu_estudios_list = ['Otro', 'Secundario','Terciario/Técnico', 'Universitario', 'Posgrado', 'Master', 'Doctorado']

edu_totales['nivel_estado'] = edu_totales['estado'].apply(lambda x: edu_estado_list.index(x)).astype('int8')
edu_totales['nivel_estudios'] = edu_totales['estudios'].apply(lambda x: edu_estudios_list.index(x)).astype('int8')

edu_totales.head()

,idpostulante,estudios,estado,nivel_estado,nivel_estudios
0,NdJl,Posgrado,En Curso,1,4
1,8BkL,Universitario,En Curso,1,3
2,1d2B,Universitario,En Curso,1,3
3,NPBx,Universitario,En Curso,1,3
4,NPBx,Master,En Curso,1,5


In [16]:
edu_totales[edu_totales['idpostulante']=='5vmKGM']

,idpostulante,estudios,estado,nivel_estado,nivel_estudios
48497,5vmKGM,Universitario,En Curso,1,3
50630,5vmKGM,Terciario/Técnico,Graduado,2,2
51662,5vmKGM,Universitario,Abandonado,0,3
51663,5vmKGM,Terciario/Técnico,Abandonado,0,2
169914,5vmKGM,Doctorado,Graduado,2,6


In [17]:
#Ordeno por niveles de importancia, primero por estado, luego por estudio

edu_totales = edu_totales.sort_values(['nivel_estado','nivel_estudios'], ascending=[True, True])
edu_totales[edu_totales['idpostulante']=='5vmKGM']

,idpostulante,estudios,estado,nivel_estado,nivel_estudios
51663,5vmKGM,Terciario/Técnico,Abandonado,0,2
51662,5vmKGM,Universitario,Abandonado,0,3
48497,5vmKGM,Universitario,En Curso,1,3
50630,5vmKGM,Terciario/Técnico,Graduado,2,2
169914,5vmKGM,Doctorado,Graduado,2,6


In [18]:
#Ante duplicados me quedo con el de nivel de importancia de estado mas alto, y dentro de ellos, con el de nivel de estudio mas alto. Esto es para evitar borrar 
#datos de personas que solo tengan estudios en curso o abandonados.

edu_totales = edu_totales.drop_duplicates(subset=['idpostulante'], keep='last')
edu_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 7621 to 385512
Data columns (total 5 columns):
idpostulante      447909 non-null object
estudios          447909 non-null category
estado            447909 non-null category
nivel_estado      447909 non-null int8
nivel_estudios    447909 non-null int8
dtypes: category(2), int8(2), object(1)
memory usage: 8.5+ MB


In [19]:
edu_totales[edu_totales['idpostulante']=='5vmKGM']

,idpostulante,estudios,estado,nivel_estado,nivel_estudios
169914,5vmKGM,Doctorado,Graduado,2,6


In [20]:
edu_totales.drop(columns=['nivel_estado','nivel_estudios'], inplace=True)

In [21]:
edu_totales.sort_values(by=['idpostulante'], ascending=True, inplace=True)
edu_totales.reset_index(drop=True, inplace=True)
edu_totales.head()

,idpostulante,estudios,estado
0,0z5Dmrd,Universitario,Graduado
1,0z5JW1r,Terciario/Técnico,Graduado
2,0z5VvGv,Secundario,Graduado
3,0zB01pE,Terciario/Técnico,Graduado
4,0zB026d,Terciario/Técnico,Graduado


- Concateno todos los detalles de avisos

In [22]:
avisos_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv'), pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')])
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45969 entries, 0 to 337
Data columns (total 11 columns):
idaviso                 45969 non-null int64
idpais                  45969 non-null int64
titulo                  45969 non-null object
descripcion             45969 non-null object
nombre_zona             45969 non-null object
ciudad                  225 non-null object
mapacalle               3444 non-null object
tipo_de_trabajo         45969 non-null object
nivel_laboral           45634 non-null object
nombre_area             45969 non-null object
denominacion_empresa    45955 non-null object
dtypes: int64(2), object(9)
memory usage: 4.2+ MB


In [23]:
avisos_totales.drop(columns=['ciudad','mapacalle','descripcion','idpais'], inplace=True)
avisos_totales['nivel_laboral'] = avisos_totales['nivel_laboral'].astype('category')
avisos_totales['tipo_de_trabajo'] = avisos_totales['tipo_de_trabajo'].astype('category')
avisos_totales['nombre_zona'] = avisos_totales['nombre_zona'].astype('category')
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45969 entries, 0 to 337
Data columns (total 7 columns):
idaviso                 45969 non-null int64
titulo                  45969 non-null object
nombre_zona             45969 non-null category
tipo_de_trabajo         45969 non-null category
nivel_laboral           45634 non-null category
nombre_area             45969 non-null object
denominacion_empresa    45955 non-null object
dtypes: category(3), int64(1), object(3)
memory usage: 1.9+ MB


In [24]:
avisos_totales.drop_duplicates(subset=['idaviso'], keep='last', inplace=True)
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 7 columns):
idaviso                 25288 non-null int64
titulo                  25288 non-null object
nombre_zona             25288 non-null category
tipo_de_trabajo         25288 non-null category
nivel_laboral           24953 non-null category
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
dtypes: category(3), int64(1), object(3)
memory usage: 1.0+ MB


In [25]:
avisos_totales.head()

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL
8,9240880,"Productores Asesores Independiente, para venta...",Capital Federal,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga
19,1110185164,Administrativo de Recepción,Gran Buenos Aires,Full-time,Junior,Administración,Unión Personal
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),Gran Buenos Aires,Full-time,Junior,Distribución,Asoko Tempo SA
32,1111034024,Vendedores para venta de medicina pre paga y o...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,JELS SRL


- Concateno todas la info de edad y sexo

In [26]:
edad_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')])
edad_totales.drop_duplicates(inplace=True)
edad_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 281384
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    478699 non-null object
sexo               505382 non-null object
dtypes: object(3)
memory usage: 15.4+ MB


In [27]:
edad_totales['sexo'] = edad_totales['sexo'].apply(lambda x: 'NO_DECLARA' if (x=='0.0') else x)
edad_totales['sexo'] = edad_totales['sexo'].astype('category')
edad_totales['fechanacimiento'] = pd.to_datetime(edad_totales['fechanacimiento'], errors='coerce', format='%Y-%m-%d')
edad_totales.fillna(inplace=True, value={'fechanacimiento': datetime.strptime('1990-01-01', '%Y-%m-%d')})
edad_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 281384
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    505382 non-null datetime64[ns]
sexo               505382 non-null category
dtypes: category(1), datetime64[ns](1), object(1)
memory usage: 12.0+ MB


In [28]:
def calculate_age(birth_date):
    return (datetime.now().year - birth_date.year - ((datetime.now().month, datetime.now().day) < (birth_date.month, birth_date.day))) # El ultimo termino resta un anio si cumple despues del dia y mes de referencia

edad_totales['edad'] = edad_totales['fechanacimiento'].apply(calculate_age)
edad_totales['edad'] = edad_totales['edad'].apply(lambda x: 28 if (x<18 or x>80) else x).astype('int8')
edad_totales.drop(columns=['fechanacimiento'], inplace=True)

edad_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 281384
Data columns (total 3 columns):
idpostulante    505382 non-null object
sexo            505382 non-null category
edad            505382 non-null int8
dtypes: category(1), int8(1), object(1)
memory usage: 8.7+ MB


In [29]:
edad_totales['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25943
Name: sexo, dtype: int64

In [30]:
edad_totales.sort_values(by=['edad'], ascending=False, inplace=True)
edad_totales.drop_duplicates(subset=['idpostulante'], keep='first', inplace=True)
edad_totales.sort_values(by=['idpostulante'], ascending=True, inplace=True)
edad_totales.reset_index(drop=True, inplace=True)
edad_totales.head()

,idpostulante,sexo,edad
0,0z5Dmrd,MASC,52
1,0z5JW1r,MASC,47
2,0z5VvGv,MASC,30
3,0zB01pE,MASC,28
4,0zB026d,FEM,40


In [31]:
edad_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504407 entries, 0 to 504406
Data columns (total 3 columns):
idpostulante    504407 non-null object
sexo            504407 non-null category
edad            504407 non-null int8
dtypes: category(1), int8(1), object(1)
memory usage: 4.8+ MB


- Concateno todas las postulaciones

In [32]:
post_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')])
post_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311264 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    object
dtypes: int64(1), object(2)
memory usage: 253.6+ MB


In [33]:
post_totales.drop_duplicates(inplace=True)
post_totales['fechapostulacion'] = pd.to_datetime(post_totales['fechapostulacion'])
post_totales.sort_values(by=['fechapostulacion'], ascending=True, inplace=True)
post_totales.drop_duplicates(subset=['idaviso','idpostulante'], keep='first', inplace=True)
post_totales.sort_values(by=['idpostulante'], ascending=True, inplace=True)
post_totales.reset_index(drop=True, inplace=True)
post_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 151.1+ MB


In [34]:
post_totales.head()

,idaviso,idpostulante,fechapostulacion
0,1112384041,0z5Dmrd,2018-03-28 16:24:37
1,1112420060,0z5Dmrd,2018-03-28 16:47:11
2,1112428361,0z5JW1r,2018-04-01 07:26:12
3,1112309728,0z5JW1r,2018-03-02 03:49:53
4,1112292169,0z5JW1r,2018-02-19 05:12:24


In [35]:
#post_totales.to_csv('post_totales.csv', index=False)

- Mergeo las postulaciones totales con las features

In [36]:
post_totales_complete = post_totales.merge(ultima_vista, on=['idaviso', 'idpostulante'], how='left')
post_totales_complete = post_totales_complete.merge(vistas_totales_count, on=['idaviso', 'idpostulante'], how='left')
post_totales_complete['veces_visto'] = post_totales_complete['veces_visto'].fillna(value=1).astype('int64')
post_totales_complete = post_totales_complete.merge(edu_totales, on='idpostulante', how='left')
post_totales_complete = post_totales_complete.merge(edad_totales, on='idpostulante', how='left')
post_totales_complete = post_totales_complete.merge(avisos_totales, on='idaviso', how='left')
post_totales_complete.insert(loc=0, column='sepostulo', value=1)
post_totales_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 16 columns):
sepostulo               int64
idaviso                 int64
idpostulante            object
fechapostulacion        datetime64[ns]
ultima_vista            datetime64[ns]
veces_visto             int64
estudios                category
estado                  category
sexo                    category
edad                    int8
titulo                  object
nombre_zona             category
tipo_de_trabajo         category
nivel_laboral           category
nombre_area             object
denominacion_empresa    object
dtypes: category(6), datetime64[ns](2), int64(3), int8(1), object(4)
memory usage: 547.9+ MB


In [37]:
post_totales_complete.head()

,sepostulo,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,estudios,estado,sexo,edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1,1112384041,0z5Dmrd,2018-03-28 16:24:37,2018-03-28 15:58:15,1,Universitario,Graduado,MASC,52,Gerente de RRHH- Para Importante Cadena de Far...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Recursos Humanos,PilarArtigas
1,1,1112420060,0z5Dmrd,2018-03-28 16:47:11,2018-03-28 16:25:03,1,Universitario,Graduado,MASC,52,Coordinador/a de RRHH,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Recursos Humanos,Importante Empresa
2,1,1112428361,0z5JW1r,2018-04-01 07:26:12,2018-04-01 07:23:35,2,Terciario/Técnico,Graduado,MASC,47,Gerente Nacional de Operaciones de Tiendas - R...,Gran Buenos Aires,Full-time,Gerencia / Alta Gerencia / Dirección,Ventas,Complement Group (holding)
3,1,1112309728,0z5JW1r,2018-03-02 03:49:53,NaT,1,Terciario/Técnico,Graduado,MASC,47,Asesor de Franquicias Gastronómicas,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Gastronomia,Grupo Myth
4,1,1112292169,0z5JW1r,2018-02-19 05:12:24,NaT,1,Terciario/Técnico,Graduado,MASC,47,Gerente de Operaciones de Franquicias ( Zona N...,Gran Buenos Aires,Full-time,Gerencia / Alta Gerencia / Dirección,Desarrollo de Negocios,Costumbres Argentinas


In [38]:
#post_totales_complete.to_csv('post_totales_complete.csv', index=False)

- Mergeo las no-postulaciones totales con las features

In [39]:
no_post_totales = pd.read_csv('no_post_totales.csv')

In [42]:
no_post_totales_complete = no_post_totales.merge(ultima_vista, on=['idaviso', 'idpostulante'], how='left')
no_post_totales_complete = no_post_totales_complete.merge(vistas_totales_count, on=['idaviso', 'idpostulante'], how='left')
no_post_totales_complete['veces_visto'] = no_post_totales_complete['veces_visto'].fillna(value=0).astype('int64')
no_post_totales_complete = no_post_totales_complete.merge(edu_totales, on='idpostulante', how='left')
no_post_totales_complete = no_post_totales_complete.merge(edad_totales, on='idpostulante', how='left')
no_post_totales_complete = no_post_totales_complete.merge(avisos_totales, on='idaviso', how='left')
no_post_totales_complete.insert(loc=0, column='sepostulo', value=0)
no_post_totales_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6604534 entries, 0 to 6604533
Data columns (total 16 columns):
sepostulo               int64
idaviso                 int64
idpostulante            object
fechapostulacion        float64
ultima_vista            datetime64[ns]
veces_visto             int64
estudios                category
estado                  category
sexo                    category
edad                    int8
titulo                  object
nombre_zona             category
tipo_de_trabajo         category
nivel_laboral           category
nombre_area             object
denominacion_empresa    object
dtypes: category(6), datetime64[ns](1), float64(1), int64(3), int8(1), object(4)
memory usage: 548.0+ MB


In [43]:
no_post_totales_complete.head()

,sepostulo,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,estudios,estado,sexo,edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,0,1112320236,0z5Dmrd,NaN,NaT,0,Universitario,Graduado,MASC,52,Cadete Administrativo,Capital Federal,Full-time,Otro,Cadetería,Geosistemas S.R.L.
1,0,1112216861,0z5Dmrd,NaN,NaT,0,Universitario,Graduado,MASC,52,Asistente Administrativo Z/CABA,Gran Buenos Aires,Part-time,Junior,Recepcionista,Pullmen Servicios Empresarios S.A.
2,0,1112262555,0z5Dmrd,NaN,NaT,0,Universitario,Graduado,MASC,52,OPERADORES TELEFÓNICOS DE EMPRESAS VINCULADAS,Gran Buenos Aires,Part-time,Senior / Semi-Senior,Atención al Cliente,BBVA Francés
3,0,1112349612,0z5Dmrd,NaN,NaT,0,Universitario,Graduado,MASC,52,Soporte Técnico TI,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Soporte Técnico,Allkom S.A.
4,0,1112326950,0z5Dmrd,NaN,NaT,0,Universitario,Graduado,MASC,52,M- Cajeros y repositores zona Caba,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Atención al Cliente,Grupo Gestión


In [44]:
#no_post_totales_complete.to_csv('no_post_totales_complete.csv', index=False)

- Mergeo el archivo de testing de 100k con las feautres asi ya lo tenemos completo

In [53]:
test_final_100k = pd.read_csv('test_final_100k.csv')
test_final_100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
id              100000 non-null int64
idaviso         100000 non-null int64
idpostulante    100000 non-null object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [54]:
test_final_100k['fechapostulacion'] = ''
test_final_100k = test_final_100k.merge(ultima_vista, on=['idaviso', 'idpostulante'], how='left')
test_final_100k = test_final_100k.merge(vistas_totales_count, on=['idaviso', 'idpostulante'], how='left')
test_final_100k['veces_visto'] = test_final_100k['veces_visto'].fillna(value=0).astype('int64')
test_final_100k = test_final_100k.merge(edu_totales, on='idpostulante', how='left')
test_final_100k = test_final_100k.merge(edad_totales, on='idpostulante', how='left')
test_final_100k = test_final_100k.merge(avisos_totales, on='idaviso', how='left')
test_final_100k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 16 columns):
id                      100000 non-null int64
idaviso                 100000 non-null int64
idpostulante            100000 non-null object
fechapostulacion        100000 non-null object
ultima_vista            34161 non-null datetime64[ns]
veces_visto             100000 non-null int64
estudios                94388 non-null category
estado                  94388 non-null category
sexo                    100000 non-null category
edad                    100000 non-null int8
titulo                  100000 non-null object
nombre_zona             100000 non-null category
tipo_de_trabajo         100000 non-null category
nivel_laboral           95569 non-null category
nombre_area             100000 non-null object
denominacion_empresa    99988 non-null object
dtypes: category(6), datetime64[ns](1), int64(3), int8(1), object(5)
memory usage: 8.3+ MB


In [55]:
test_final_100k.head()

,id,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,estudios,estado,sexo,edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,0,739260,6M9ZQR,,NaT,0,Doctorado,Graduado,FEM,42,Asistente Comercial,Capital Federal,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA
1,1,739260,6v1xdL,,NaT,0,Terciario/Técnico,Graduado,MASC,30,Asistente Comercial,Capital Federal,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA
2,2,739260,ezRKm9,,NaT,0,Universitario,Graduado,FEM,36,Asistente Comercial,Capital Federal,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA
3,3,758580,1Q35ej,,NaT,0,Posgrado,Graduado,MASC,68,Oracle Financials Sr. Application Developer,Capital Federal,Full-time,Otro,Tecnologia / Sistemas,TeleTech
4,4,758580,EAN4J6,,NaT,0,Universitario,En Curso,FEM,32,Oracle Financials Sr. Application Developer,Capital Federal,Full-time,Otro,Tecnologia / Sistemas,TeleTech


In [56]:
#test_final_100k.to_csv('test_final_100k_complete.csv', index=False)